ドライブへのマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Sun Mar 30 03:15:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install flash-attn --no-build-isolation
!pip install wandb huggingface_hub
!pip install datasets

import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-fz_756hd/unsloth_fd3c159ef6d94092a8b4f9eb69766fdf
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-fz_756hd/unsloth_fd3c159ef6d94092a8b4f9eb69766fdf
  Resolved https://github.com/unslothai/unsloth.git to commit 2ff5dc1a8de1614994a275785b7b64fb4db8cb5d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import wandb
from google.colab import userdata
wandb_api_key = userdata.get('wandb_api')
hf_token = userdata.get('hf_token')
!wandb login $wandb_api_key

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [5]:
import torch
from unsloth import FastLanguageModel
from peft import peft_model

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.9 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [30]:
max_seq_length = 2048 # unslothではRoPEをサポート
dtype = torch.float16
load_in_4bit = True

In [31]:
model_id = "unsloth/Llama-3.1-8B-Instruct-bnb-4bit"
peft_model_id = "Yuichi1218/Tetun-llama3-chat-CPT"

In [32]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    #dtype = dtype,
    load_in_4bit = load_in_4bit,
    trust_remote_code = True,
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [10]:
from datasets import load_from_disk
dataset = load_from_disk('/content/drive/MyDrive/Colab Notebooks/Tetun-LLM/dataset/peace_corpus_tetun')

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    return { "text" : [example + EOS_TOKEN for example in examples["text"]] }
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/6227 [00:00<?, ? examples/s]

Map:   0%|          | 0/692 [00:00<?, ? examples/s]

In [17]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # LoRAのランク
    target_modules =  ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "embed_tokens", "lm_head",],
    lora_alpha = 16,
    lora_dropout = 0.2,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
    max_seq_length = max_seq_length,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [18]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

In [19]:
training_args = UnslothTrainingArguments(
    run_name = "llama3-chat-8B-CPT", # wandbのプロジェクトのRUN名
    per_device_train_batch_size = 4, # デバイスごとのトレーニングバッチサイズ
    gradient_accumulation_steps = 64, # 勾配を更新する前にステップを積み重ねる回数
    optim = "adamw_8bit", # オプティマイザの設定
    # weight_decay = 0.01, # 指定する場合
    num_train_epochs = 1, # エポック数
    logging_steps = 5, # ログを出力するステップ間隔
    warmup_steps = 5, # 学習率のウォームアップステップ数
    save_strategy = 'steps',
    save_steps = 40, # モデルを保存するステップ間隔
    save_total_limit = 2, # 保存しておくcheckpointの数
    max_steps = -1, # トレーニングの最大ステップ数
    learning_rate = 3e-5, # 学習率
    embedding_learning_rate = 3e-6, # embed層の学習率（他の2-10x小さい値）
    fp16 = not is_bfloat16_supported(), # GPUによってはbf16が使えないときはこっちがTrue
    bf16 = is_bfloat16_supported(), # bf16が使えるときはTrue
    group_by_length=True,
    seed = 3407,
    output_dir = 'drive/MyDrive/Result_CPT', # モデルの保存場所（ここではGoogleDriveに接続したとき）
    report_to = "wandb", # ログの送信先 ("wandb"/"tensorboard"など)
    auto_find_batch_size = True, # バッチサイズの自動設定
)

In [20]:
trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    max_seq_length = max_seq_length,
    dataset_text_field = "text", # datasetのテキストが入ったkeyを指定
    packing = False,
    args = training_args,
)

Map:   0%|          | 0/6227 [00:00<?, ? examples/s]

In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 6,227 | Num Epochs = 1 | Total steps = 24
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 64
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 64 x 1) = 256
 "-____-"     Trainable parameters = 1,386,217,472/8,000,000,000 (17.33% trained)


Unsloth: Setting lr = 3.00e-06 instead of 3.00e-05 for embed_tokens.
Unsloth: Setting lr = 3.00e-06 instead of 3.00e-05 for lm_head.


AssertionError: Backwards requires embeddings to be bf16 or fp16

In [ ]:
model = model.to(dtype=torch.bfloat16) # 念の為、bf16に変換する（必要かどうかは未検証）
model.push_to_hub_merged(
    "Tetun-llama3-8B-chat-CPT", # 保存するレポジトリ名
    tokenizer=tokenizer, # tokenizerも保存する場合（基本入れますわなぁ・・・）
    save_method="merged_16bit", # LoRAモデルとベースモデルをマージして保存するとき
    token=hf_token, # 保存するアカウントのアクセストークン
    private=False # private設定の時はTrue、publicの時はFalse
)